# Проект: Оценка результатов A/B-теста

## Описание проекта.

Цель проекта: провести оценку результатов A/B-теста. 
Задачи проекта:
1. Оценить корректность проведения теста.
а. проверить пересечение тестовой аудитории с конкурирующим тестом;
б. сделать проверку на совпадение теста и маркетинговых событий;
в. проверить соблюдение временнЫх границ теста;
г. проверить корректность подбора аудитории теста.
2. Проанализировать результат теста.

Источники данных:
1. Техническое задание.
2. Четыре таблицы, содержащие календарь маркетинговых событий, данные о пользователях, данные о событиях и данные о разбивке аудитории на группы по тестам.

Структура проекта:
1. [Изучение данных.](#start)
2. [Предобработка данных.](#preprocessing)
3. [Оценка корректности проведения теста.](#asset)
4. [Исследовательский анализ данных.](#eda)
5. [Оценка результатов A/B-тестирования..](#a/b_testing)
6. [Выводы.](#outcome)

In [9]:
#Библиотеки для работы с данными
import pandas as pd
import numpy as np

from scipy import stats as st
import math as mth

#модуль datetime для работы с датами
from datetime import datetime as dt
from datetime import date

#Библиотеки для визуализации данных
import seaborn as sns
import matplotlib.pyplot as plt
from plotly import graph_objects as go

## Изучение данных.
<a id="start"></a>

In [10]:
marketing_calender = pd.read_csv('C:/datasets/ab_project_marketing_events.csv')

In [11]:
display(marketing_calender)
marketing_calender.info()

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [12]:
marketing_calender['regions'].unique()

array(['EU, N.America', 'EU, CIS, APAC, N.America', 'N.America', 'APAC',
       'EU, CIS, APAC', 'CIS'], dtype=object)

Маркетинговые мероприятия охватывают четыре региона:Европейский Союз, Северная Америка, Азиатско-Тихоокеанский регион и СНГ. Часть мероприятий охватывает два, три или четыре региона одновременно.

In [13]:
new_user_data = pd.read_csv('C:/datasets/final_ab_new_users.csv')

In [14]:
display(new_user_data.head(10))
new_user_data.info()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
5,137119F5A9E69421,2020-12-07,N.America,iPhone
6,62F0C741CC42D0CC,2020-12-07,APAC,iPhone
7,8942E64218C9A1ED,2020-12-07,EU,PC
8,499AFACF904BBAE3,2020-12-07,N.America,iPhone
9,FFCEA1179C253104,2020-12-07,EU,Android


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [15]:
display(new_user_data['region'].unique())
display(new_user_data['device'].unique())

array(['EU', 'N.America', 'APAC', 'CIS'], dtype=object)

array(['PC', 'Android', 'iPhone', 'Mac'], dtype=object)

В тестах (мы пока не знаем, сколько их) принимают участие жители четырёх регионов: Европейский Союз, Северная Америка, Азиатско-Тихоокеанский регион и СНГ, использующие четыре типа устройств: персональные компьютеры (здесь возможны Windows - более вероятно, и Linux - менее вероятно), мобильные телефоны на OS Android, мобильные телефоны на iOS и персональные компьютеры (более вероятно - ноутбуки на MacOS).

In [16]:
user_events_data = pd.read_csv('C:/datasets/final_ab_events.csv')

In [17]:
display(user_events_data.head(10))
user_events_data.info()

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99
5,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99
6,6B2F726BFD5F8220,2020-12-07 11:27:42,purchase,4.99
7,BEB37715AACF53B0,2020-12-07 04:26:15,purchase,4.99
8,B5FA27F582227197,2020-12-07 01:46:37,purchase,4.99
9,A92195E3CFB83DBD,2020-12-07 00:32:07,purchase,4.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


In [18]:
#Проверяем уникальные значения в столбце event_name
display(user_events_data['event_name'].unique())

array(['purchase', 'product_cart', 'product_page', 'login'], dtype=object)

Столбец event_name таблицы user_events_data содержит четыре типа событий: вход в систему, посещение страницы с определённым типом продукта, товара в корзину, покупка.

In [19]:
test_participants = pd.read_csv('C:/datasets/final_ab_participants.csv')

In [20]:
display(test_participants.head(10))
test_participants.info()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test
5,4FF2998A348C484F,A,recommender_system_test
6,7473E0943673C09E,A,recommender_system_test
7,C46FE336D240A054,A,recommender_system_test
8,92CB588012C10D3D,A,recommender_system_test
9,057AB296296C7FC0,B,recommender_system_test


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


In [21]:
#Проверяем уникальные значения столбца ab_test таблицы test_participants
display(test_participants['ab_test'].unique())

array(['recommender_system_test', 'interface_eu_test'], dtype=object)

В таблице содержится информация о двух тестах, из которых мы проверяем только recommender_system_test.

## Предобработка данных.
<a id="preprocessing"></a>

### Приведение данных к нужным типам.

In [22]:
#Меняем тип данных object на datetime в столбцах трёх таблиц, содержащих даты.
marketing_calender['start_dt'] = pd.to_datetime(marketing_calender['start_dt'], format = '%Y.%m.%d')
marketing_calender['finish_dt'] = pd.to_datetime(marketing_calender['finish_dt'], format = '%Y.%m.%d')
marketing_calender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes


In [23]:
new_user_data['first_date'] = pd.to_datetime(new_user_data['first_date'], format = '%Y.%m.%d')
new_user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     61733 non-null  object        
 1   first_date  61733 non-null  datetime64[ns]
 2   region      61733 non-null  object        
 3   device      61733 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.9+ MB


In [24]:
user_events_data['event_dt'] = pd.to_datetime(user_events_data['event_dt'], format = '%Y.%m.%d %H:%M:%S')
#Создаём колонку с датой, по которой будем фильтровать участников и события по времен
user_events_data['day'] = user_events_data['event_dt'].dt.strftime('%Y-%m-%d')
user_events_data['day'].unique()

array(['2020-12-07', '2020-12-08', '2020-12-09', '2020-12-10',
       '2020-12-11', '2020-12-12', '2020-12-13', '2020-12-14',
       '2020-12-15', '2020-12-16', '2020-12-17', '2020-12-18',
       '2020-12-19', '2020-12-20', '2020-12-21', '2020-12-22',
       '2020-12-23', '2020-12-24', '2020-12-25', '2020-12-26',
       '2020-12-27', '2020-12-28', '2020-12-29', '2020-12-30'],
      dtype=object)

### Проверка данных на дубликаты.

In [25]:
display(marketing_calender.duplicated().sum())

0

Маркетинговые события не дублируются.

In [26]:
display(new_user_data.duplicated().sum())
display(new_user_data['user_id'].duplicated().sum())

0

0

В данных о пользователях нет дубликатов.

In [27]:
display(user_events_data.duplicated().sum())
display(user_events_data['user_id'].duplicated().sum())

0

381614

В таблице с данными о событиях нет дубликатов строк. Обилие дубликатов в столбце user_id говорит о том, что на каждого пользователя приходится больше одного события.

In [28]:
display(test_participants.duplicated().sum())
display(test_participants['user_id'].duplicated().sum())


0

1602

Повторяющихся строк в таблице нет.
Следовательно, 1602 дубликата в столбце user_id таблицы test_participants свидетельствуют о том, что 1602 человека участвуют либо в разных тестах, либо оказались в одной в обеих группах одного теста.
Вернёмся к вопросу при проверке аудитории теста.

### Проверка данных на пропуски и выявление их характера.

In [29]:
#Создаю таблицу для анализа пропусков
user_events_data_gap = user_events_data.copy()

In [ ]:
user_events_data_gap = pd.get_dummies(user_events_data_gap, columns=['event_name'], drop_first=False)

In [ ]:
#Создаю колонку, содержащую информацию о наличии пропусков
user_events_data_gap['events_gap'] = user_events_data.details.isna() * 1
display(user_events_data_gap.head())

In [ ]:
#Группирую данные по наличию пропуска в событиях. Проверяю средние значения.
user_events_data_gap_group = user_events_data_gap.groupby('events_gap').mean().round(2).reset_index()
display(user_events_data_gap_group)

Все значения в details связаны с событием purchase (покупка) в столбце event_name. Следовательно, пропуски в колонке details появляются каждый раз, когда клиент вошёл на сайт, прошёл до корзины, но не совершил покупку. Пропуски связанны со статусом покупки в системе (совершена - не совершена). Пропуск не случаен и относится к типу пропусков MNAR (Missing Not At Random / Отсутствует не случайно). С этими пропусками ничего не нужно было бы делать, даже если бы их было не много.

## Оценка корректности проведения теста.
<a id="asset"></a>

#### Проверка на соответствие данных требованиям технического задания.

In [ ]:
#Выясняем даты начала и окончания набора участников тестирования.
temp_new_user_data = test_participants.merge(new_user_data, on = 'user_id', how = 'left')
test_new_user_data = temp_new_user_data.query('ab_test == "recommender_system_test"')
new_user_data_max = new_user_data.max()
new_user_data_min = new_user_data.min()
display(new_user_data_max)
display(new_user_data_min)
test_new_user_data['first_date'].max()
#test_new_user_data.nunique

Согласно ТЗ набор участников теста должен быть остановлен 21 декабря 2020 г., то есть, самые новые участники должны были быть набраны 20 декабря 2020 г. В реальности последние участники присоединились 23 декабря.

In [ ]:
#Выясняем дату остановки теста.
user_events_data['day'].max()

Точную дату формального прекращения теста установить не удалось. Фактически, с 31 декабря 2020 г. по 4 января 2021 г. тестирование уже не проводилось. Очевидно, "затишье" было вызвано праздниками. Данное обстоятельство делает результаты тестирования некорректными.

In [ ]:
#Создаём таблицу, включающую данные о пользователях, об активности и о тестах пользователей для recommender_system_test
#total_data_table = temp_new_user_data.merge(user_events_data, on = 'user_id', how = 'left')
#Находим количество уникальных user_id в таблице
total_number_unique_user_id = temp_new_user_data.nunique(axis = 0)
display('Общее количество участников тестов: ', total_number_unique_user_id['user_id'])

#Находим общее количество уникальных user_id всех европейцев, зарегистрировавшихся
#по 21 декабря 2020 г., включительно.
total_data_table_eu = temp_new_user_data.query('region == "EU" and first_date <= "2020-12-21"').nunique(axis = 0)
display('Общее количество участников тестов из Евросоюза: ', total_data_table_eu['user_id'])

#Находим общее количество уникальных user_id всех европейцев в тесте recommender_system_test
#data_table_recommender_system_test = total_data_table.query('ab_test == "recommender_system_test"')
recommender_system_test_eu = test_new_user_data.query('region == "EU"').nunique(axis = 0)
display('Количество участников теста recommender_system_test из Евросоюза: ', recommender_system_test_eu['user_id'])

#Находим долю европейцев, участвовавших в recommender_system_test от числа всех зарегистрированных европейцев.
eu_users_share = recommender_system_test_eu['user_id'] / total_data_table_eu['user_id']
display('Доля европейцев в тесте recommender_system_test от общего количества участников теста: ', eu_users_share)

#Находим общее количество уникальных user_id всех участников recommender_system_test
user_recommender_system_test_number = test_new_user_data.nunique(axis = 0)
display('Общее количество участников теста recommender_system_test', user_recommender_system_test_number['user_id'])
total_data_table_eu

Всего пользователей в таблице 16666. Пользователей из Евросоюза - 16316. 6351 пользователь из Евросоюза участвовал в тесте recommender_system_test, что составило 41.47% от числа европейцев, зарегистрировавшихся по 21 декабря включительно. В 2.76 раза больше, чем определено в ТЗ. Это должно серьёзно сказаться на репрезентативности теста, и, в конечном итоге, на его результатах.

Ожидаемое количество участников в 6000 человек. Набрано для теста 6700. Тот факт, что активность была только до 30 декабря включительно, учтём в процессе EDA. Ожидается, что количество участников ещё уменьшится. Здесь явное несоответствий требованиям ТЗ.

In [ ]:
recommender_system_test_eu['user_id']/len(new_user_data.query('first_date <= "2020-12-21" & region == "EU"'))

#### Проверка времени проведения теста.

In [ ]:
#Выявляем совпадение теста с маркетинговой активностью.
#marketing_calender_test = marketing_calender.query('start_dt >= "2020-12-07"')
#display(marketing_calender_test)

test_period = pd.Interval(pd.Timestamp('2020-12-07 00:00:00'),
                        pd.Timestamp('2021-01-04 00:00:00'))

marketing_calender_test2 = []

for i in range(len(marketing_calender)):
    event_interval = pd.Interval(pd.Timestamp(marketing_calender['start_dt'][i]),
                                pd.Timestamp(marketing_calender['finish_dt'][i]))
    is_event_overlaping = test_period.overlaps(event_interval)
    if is_event_overlaping:
        marketing_calender_test2.append(marketing_calender.loc[i])
        
display(pd.DataFrame(marketing_calender_test2))
        

Два маркетинговых события, охватывающих три больших региона совпадают с периодом тестирования.Сами эти события так же накладываются друг на друга, что делает период с 25 декабря 2020 г. по 4 января 2021  неудачным для тестирования, поскольку активность клиентов в этот период отличается от нормальной. Для периода с 30 декабря 2020 г. по 3 января 2021 г. ситуация ещё менее подходящая, поскольку в это время тест пересекается с двумя маркетинговыми событиями одновременно. 

#### Проверка аудитории теста.

In [ ]:
#Проверяю аудиторию на пересечение двух тестов.
#Создаём таблицу с дубликатами.
cross_test = test_participants[test_participants['user_id'].duplicated() == True]

#Проходим по таблице срезом.
check_result_system_test = cross_test.query('ab_test == "recommender_system_test"')
check_result_eu_interface = cross_test.query('ab_test == "interface_eu_test"')
display(check_result_system_test['user_id'].count())
display(check_result_eu_interface['user_id'].count())


В таблице с участниками тестов (test_participants) 1602 дубликата. Срезом query() по таблице с дубликатими (cross_test) мы получаем либо 1602 человека участвующих в параллельном тесте, либо ноль, если делаем срез по интересующему нас тесту. Из этого следует, что все 1602 дубликата в колонке user_id в двух тестах одновременно.
Это некритично, если параллельный тест касается другого продукта или если во втором тесте участник находится в той же группе.

In [ ]:
#Проверяем участников теста на участие в двух группах одновременно
#Проверяю аудиторию на участие в двух группах.
#Создаём таблицу с дубликатами для всех участников.
cross_group_total = test_participants[test_participants['user_id'].duplicated() == True]

#Выясняем, в какую группу в recommender_system_test попали участники теста interface_eu_test.
cross_test_groups = cross_group_total.merge(test_participants, on = 'user_id', how = 'left')
#display(cross_test_groups)
result = cross_test_groups.query('ab_test_x == "interface_eu_test" and ab_test_y == "recommender_system_test"')
#display(result)
result_user_count_a = result.query('group_x == "A"')
result_user_count_b = result.query('group_x == "B"')
display('Число участников теста recommender_system_test, попавших в контрольную группу теста  interface_eu_test', result_user_count_a['user_id'].count())
display('Число участников теста recommender_system_test, попавших в тестовую группу теста interface_eu_test ', result_user_count_b['user_id'].count())


#Ищем пересечения по группам внутри теста recommender_system_test.
#Использую ранее сделанное объединение таблиц участников теста recommender_system_test и данных пользователей.
duplicated_group = test_new_user_data[test_new_user_data['user_id'].duplicated() == True]
duplicated_group_count = duplicated_group.nunique()
display(duplicated_group)
display(duplicated_group_count)
#Для надёжности проверяю, есть ли значения в предполагаемой таблице с дубликатами user_id по группам.
duplicated_group_a = duplicated_group.query('group == "A"')
duplicated_group_b = duplicated_group.query('group == "B"')
display(duplicated_group_a)
display(duplicated_group_b)
#Сравниваю общее количество user_id в таблице и количество уникальных значений в user_id
#duplicated_id_count = test_new_user_data['user_id']
#duplicated_id = data_table_recommender_system_test['user_id'].nunique()
#duplicate_test = data_table_recommender_system_test.nunique(axis = 0)
#display('Общее количество user_id: ', duplicated_id_count)
#display('Количество уникальных user_id: ', duplicated_id)
#duplicate_test

Проверка на дубликаты и на количество дубликатов в таблице с данными участников recommender_system_test показала отсутствие пересечений по группам внутри этого теста.
Следовательно, никто в тесте recommender_system_test не участвует в двух группах одновременно.

Дубликаты по группам существуют только по причине того, что часть пользователей участвует в двух тестах.

783 участника recommender_system_test попали в тестовую группу теста interface_eu_test. Следовательно, результаты recommender_system_test могут быть искажены.

In [ ]:
#Строим общую таблицу.
full_table = test_participants.merge(new_user_data, on = 'user_id', how = 'left')
full_table_result = full_table.merge(user_events_data, on = 'user_id', how = 'left')
#display(full_table_result)

#Фильтруем данные по участию в тесте.
#Фильтруем события старше 14 дней.
recommender_system_test_time_delta = full_table_result.drop(full_table_result[full_table_result['event_dt'] >= (full_table_result['first_date'] + pd.Timedelta(14, 'D'))].index)
#display(recommender_system_test_table)
#Учитываем только участников recommender_system_test.
recommender_system_test = recommender_system_test_time_delta.query('ab_test == "recommender_system_test"')
# считаем общее количество участников, подходящих по времени
participants_count = recommender_system_test['user_id'].nunique()
display('Всего участников, соответствующих периоду, указанному в ТЗ: ' ,participants_count)

#Убираем неактивных пользователей
non_active_count_a = recommender_system_test.query('event_dt == "NaT" and group == "A"')
non_active_count_b = recommender_system_test.query('event_dt == "NaT" and group == "B"')
display('Количество неактивных участников в группе А: ', non_active_count_a['user_id'].nunique())
display('Количество неактивных участников в группе В: ', non_active_count_b['user_id'].nunique())

#Таблица активных участников
filter_full_table_result = recommender_system_test.query('event_dt != "NaT"')
#display(filter_full_table_result)
display('Количество активных участников в тесте: ', filter_full_table_result['user_id'].nunique())

#Определяем количество активных участников в каждой группе с учётом соблюдения 14-дневного срока
filter_participants_a = filter_full_table_result.query('group == "A"')
filter_participants_b = filter_full_table_result.query('group == "B"')
display('Количество активных участников в группе А: ',filter_participants_a['user_id'].nunique())
display('Количество активных участников в группе В: ',filter_participants_b['user_id'].nunique())


Всего в тесте 6701 человек. Из них 3026 - неактивных.
Из них в группе А 1077 и в группе В 1949 человека.
Активных участников в тесте всего 3675 человека, вместо необходимых 6000.
В группе А остаётся 2747 человек, и в группе В - 928 человек.
Распределение между группами в любом случае остаётся неравномерным: группа А составляет 74.75% от числа всех участников.

In [ ]:
#Проверяем деление трафика по устройствам в каждой группе.
device_number_a = filter_participants_a.query('device == "PC" or  device == "Mac"')
device_number_b = filter_participants_b.query('device == "PC" or  device == "Mac"')
#Делим совокупное количество владельцев десктопных устройств на общее количество людей для каждой из двух груп
display(device_number_a['device'].count() / filter_participants_a['user_id'].count())
display(device_number_b['device'].count() / filter_participants_b['user_id'].count())


Владельцы компьютеров и ноутбуков среди активных участников составляют по группам 35.94% для группы А и 30.14% для группы В. Трафик  между мобильными и десктопными устройствами распределён неравномерно, хотя различие между группами по этому показателю невелико. Доля владельцев десктопных устройств в тесте, очевидно отражает их реальную долю от общего количества людей, пользующихся интенетом.

### Промежуточный вывод о качестве исходных данных.

При проведении тестирования необходимо учитывать сложный состав данных, которые нельзя назвать чистыми.
Пропуски есть только в таблице с активностью. Они являются логичным следствием принципы учёта активности и анализу не мешают.
В таблице участников тестов. Присутствуют в большом количестве(1602) дубликаты user_id. Это связано с тем, что пользователи участвовали в двух тестах. Часть пользователей нашего теста участвовала в конкурирующем в составе тестовой группы, что может оказать влияние на результат.
Других дубликатов, включая пересечение по группам внутри нашего теста, нет. Это очень хорошо.
Данные по активности обрываются после 30 декабря, что сильно затрудняет анализ результатов теста.
Данные об участниках можно разделить на три уровня по критерию активности в тесте. Вместо 6000 человек для этого А/В-теста набрано 6701 (1-й уровень); участвовало в тесте (совершали события) 4673 человек (2-й уровень); участвовали все 14 дней 2272 активного участника совокупно, в обеих группах (3-уровень).
Группы представленные изначально отличаются по количеству очень сильно в 2.66 раза. Разница численности групп активных пользователей - 2.36 раза. Группа В в обоих случаях меньше. Это не оказывает принципиального влияния на качество теста.
Доля жителей Европы, привлечённых в тест, от общего числа зарегистрированных за время теста европейцев, в реальности составила 38.92% зарегистрированных пользователей из Европы, вместо оговоренных 15%, что превышает условие в ТЗ в 2.5 раза.
Данные так же указывают на совпадение теста по времени с двумя маркетинговыми мероприятиями, которые могут повлиять на поведение пользователей и, как следствие, на результат теста.

Принципиальное влияние на исход теста могут оказать:
неполные данные по длительности теста;
доля европейцев от общего числа зарегистрированных европейцев;
участие пользователей в тестовой группе конкурирующего теста;
совпадение теста с маркетинговыми событиями.

 ## Исследовательский анализ данных.
 <a id="eda"></a>

### Сравнение распределений количества событий на пользователя для обеих групп.

In [ ]:
#Строим распределение количества событий по пользователям.

#Распределение количества событий по пользователям для группы А:

events_by_users_a = (
    filter_participants_a.groupby('user_id', as_index=False)
    .agg({'event_name': 'count'})
)

#Распределение количества событий по пользователям для группы В:
events_by_users_b = (
    filter_participants_b.groupby('user_id', as_index=False)
    .agg({'event_name': 'count'})
)


#Строим графики
ax = sns.histplot(events_by_users_a['event_name'], bins = 25, color = 'blue')
sns.histplot(events_by_users_b['event_name'], bins = 25, color="orange", alpha = 0.7)
plt.title('Количество событий на участника теста.')
ax.set(xlabel='Количество событий', ylabel='Количество участников')
ax.legend(['Группа А', 'Группа В'])
plt.show()


sns.boxplot(x = 'event_name', data = events_by_users_a)
plt.title('Активность участников группы А')
plt.xlabel('Количество событий для участников группы А')
plt.show()
sns.boxplot(x = 'event_name', data = events_by_users_b, color="orange")
plt.title('Активность участников группы В')
plt.xlabel('Количество событий для участников группы В')

plt.show()


In [ ]:
display(np.percentile(events_by_users_a['event_name'], [95, 97.5, 99]))
display(np.percentile(events_by_users_b['event_name'], [95, 97.5, 99]))

Здесь учитываем только тех участников, которые были активны 14 дней (или по которым сохранились данные за 14 дней).
Распределение активности обеих групп близко к нормальному.
Активность в совершении событий для группы А несколько выше. Медианное значение событий на человека в контрольной группе - 6, в тестовой - 4. Максимальная статистически значимая величина количества событий на человека - 16 для группы А и 12 для группы В.
Количественно выбросы в группах одинаковые, но в группе В присутствуют более заметные выбросы.
Участники группы В менее активны.

### Сравнение распределений число событий в выборках по дням

In [ ]:
#Создаём таблицы по выборкам

events_by_day_a = (
    filter_participants_a.groupby('day', as_index=False)
    .agg({'event_name': 'count'})
)
events_by_day_a.columns = ['day', 'event_name']

#Для группы В:
events_by_day_b = (
    filter_participants_b.groupby('day', as_index=False)
    .agg({'event_name': 'count'})
)
events_by_day_b.columns = ['day', 'event_name']


#Строим графики
plt.figure(figsize=(15,7))
plt.barh(events_by_day_a['day'], events_by_day_a['event_name'], color='blue', alpha=0.45, label='Группа А')
plt.barh(events_by_day_b['day'], events_by_day_b['event_name'], color='orange', alpha=0.75, label='Группа В')
plt.xlabel('Кол-во событий')
plt.ylabel('ДАТА')
#plt.title('Распределение событий по дням')
plt.legend()
plt.show()

События по дням в группе А распределены близко к нормальному. В группе В - два пика, в начале и в середине теста. 30 декабря активность резко обрывается. В этот день активность (незначительную) показали только участники группы В.
Распределение по дням показывает бОльшую активность контрольной группы, что, судя по разнице графиков, не всегда может быть объяснено только значительным численным превосходством группы А.

In [ ]:
group_participants_by_date_a = filter_participants_a.query('group == "A"').groupby('first_date')['user_id'].count()
group_participants_by_date_b = filter_participants_b.query('group == "B"').groupby('first_date')['user_id'].count()

display(group_participants_by_date_a)
display(group_participants_by_date_b)

14 декабря в группе А происходит всплеск регистраций пользователей, затем показатель снижается (после 15 декабря - постепенно). В группе бы 16 декабря происходит однодневный всплеск регистраций, на второй день происходит резкое павдение этого показателя. Показатель для группы А превосходит показатель группы В в разы. 15 декабря - более чем 10 раз.

### Изучение изменений конверсии в воронке в выборках на разных этапах.

In [ ]:
#Строим таблицы по группам для участников, которые участвовали весь срок - 14 дней
#filter_participants_a = filter_full_table_result.query('group == "A"')
#filter_participants_b = filter_full_table_result.query('group == "B"')
conversion_rate_a = filter_participants_a.groupby('event_name').agg({'user_id': 'nunique'}).sort_values(by='event_name', ascending=False).reset_index()
conversion_rate_b = filter_participants_b.groupby('event_name').agg({'user_id': 'nunique'}).sort_values(by='event_name', ascending=False).reset_index()

sorted_conversion_rate_a = conversion_rate_a.sort_values(by = 'user_id', ascending = False)
sorted_conversion_rate_b = conversion_rate_b.sort_values(by = 'user_id', ascending = False)
display(sorted_conversion_rate_a)
sorted_conversion_rate_b


Обращает на себя внимание, что в обеих товар в корзину положило меньше участников, чем совершило покупку. Это вызвано, тем, что здесь нет прямой последовательности.
С одной стороны, пользователь может использовать корзину вместо "Избранного", если оно вообще есть в данном случае. То есть, человек кладёт что-то в корзину, чтобы подумать, точно ли ему нужен этот продукт.
С другой стороны, покупатели могут воспользоваться возможностью "быстрой покупки", переходя сразу от товара к оплате. Скорее всего здесь мы сталкиваемся с людьми, которые уже точно знают, какой товар и с какими характеристиками они хотят приобрести.
В группе В залогинилось на 1 человека меньше, чем в тесте приняло активное участие. Кто-то совершал покупку без входа в личный кабинет или подобного действия.

In [ ]:
#Строим воронки для каждой группы.
sorted_conversion_rate_a = sorted_conversion_rate_a.reindex([3, 1, 2, 0])
#Воронка для группы А
fig = go.Figure(
    go.Funnel(
        y= sorted_conversion_rate_a['event_name'],
        x= sorted_conversion_rate_a['user_id'],
        
    )
    
)
fig.update_layout(title={"text": "Воронка продаж для группы А."})
fig.show()



#Воронка для группы В
sorted_conversion_rate_b = sorted_conversion_rate_b.reindex([3, 1, 2, 0])
fig = go.Figure(
    go.Funnel(
        y= sorted_conversion_rate_b['event_name'],
        x= sorted_conversion_rate_b['user_id'],
    )
)
fig.update_layout(title={"text": "Воронка продаж для группы В."})

fig.show() 


In [ ]:
#Создаём таблицы конверсии относительно login для каждой группы
sorted_conversion_rate_a['conversion'] = (sorted_conversion_rate_a['user_id'] / sorted_conversion_rate_a.iloc[0, 1] * 100).round(2)
sorted_conversion_rate_b['conversion'] = (sorted_conversion_rate_b['user_id'] / sorted_conversion_rate_b.iloc[0, 1] * 100).round(2)
display(sorted_conversion_rate_a)
sorted_conversion_rate_b

При проведении тестирования необходимо учитывать сложный состав данных, которые нельзя назвать чистыми.
Во-первых, нужно учесть, что таблицы содержат информацию по двум тестам.
Во-вторых, данные по активности обрываются после 30 декабря, так что последние участники тестировались только 10-13 дней и их активность учитывать нельзя.
В-четвёртых, хотя, вместо 6000 человек для этого А/В-теста набрано 6701, активно участвовало в тесте 3675 человек.
В-пятых, группы отличаются по количеству очень сильно. Группа А больше группы В в 2.96 раза.
В-шестых, доля жителей Европы, попавших в тест, в 2.76 раз превышает условие в ТЗ.
Наконец, данные указывают на совпадение теста по времени с двумя маркетинговыми событиями и с двумя праздниками: Рождеством (помним, что у нас большинство участников из региона Европейский Союз, добавляем к ним участников из Северной Америки) и Новым годом (что актуально на территории СНГ). Если Рождество в Европе - семейный праздник, то на период от Рождества до Трёх королей (6 января), а то и до 10 января, многие уезжают в отпуска для активного отдыха. Тест необходимо было перенести, а состав участников пополнить.

## Оценка результатов A/B-тестирования.
<a id="a/b_testing"></a>

In [ ]:
#Строим общую таблицу с конверсией для обеих групп.
result_conversion_rate = sorted_conversion_rate_a.merge(sorted_conversion_rate_b, on = 'event_name')

result_conversion_rate.rename(columns = {'user_id_x':'events_number_a', 'conversion_x':'conversion_a', 'user_id_y':'events_number_b', 'conversion_y':'conversion_b' }, inplace = True )
#Добавляем колонку с разницей конверсии.
result_conversion_rate['conversion_delta %'] = (result_conversion_rate['conversion_b'] / result_conversion_rate['conversion_a'] * 100 - 100).round(2)
result_conversion_rate

На всех этапах воронки конверсия в группе В хуже, чем в группе А.
Вместо роста по всем метрикам на 10% получили падение метрик: по конверсии в product_page  -12.93%, по конверсии в product_cart  -8.3% и по конверсии в purchase  -12.98%.
Вместо заметного роста получили заметное падение.
Мы могли бы предположить, что внесённые изменения ухудшили продукт или усложнили пользовательский опыт. Однако мы должны учитывать, что по многоим пунктам организаторам теста не удалось соблюсти не только условия ТЗ, но и общие принципы А/В-тестирования. 

### Проверка статистической разницы долей z-критерием.

Нулевая гипотеза: доли для контрольной и тестовой групп не имеют статистических различий.
Альтернативная гипотеза: доли для контрольной и тестовой групп имеют статистические различия.

In [ ]:
#Создаём функцию для проведения z-теста по четырём метрикам.

def z_test(successes, trials, alpha):
    
    alpha = .05

# Пропорция успехов в первой группе:
    p1 = successes[0]/trials[0]

# Пропорция успехов во второй группе:
    p2 = successes[1]/trials[1]
    
    display(successes[0], successes[1], trials[0], trials[1])
# Пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])

# Разница пропорций в датасетах
    difference = p1 - p2 


# Считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))

# Задаем стандартное нормальное распределение
    distr = st.norm(0, 1)  

    p_value = ((1 - distr.cdf(abs(z_value))) * 2)
    
    
    display('p-значение ', p_value)
   

    

    if p_value < alpha:
        display('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        display(
        'Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')    
        

In [ ]:
#Создаём цикл для использования функции.
alpha = .05
alpha_bonferroni = alpha/3

trials = [sorted_conversion_rate_a.iloc[0, 1], sorted_conversion_rate_b.iloc[0, 1]]

event_table = result_conversion_rate[['event_name', 'events_number_a', 'events_number_b']]

event_dict = event_table.set_index('event_name').to_dict('index')

for ev in ['product_page', 'product_cart', 'purchase']:
    successes = [event_dict[ev]["events_number_a"], event_dict[ev]["events_number_b"]]
    display(f'Тестируем событие "{ev}" / successes = {event_dict[ev]["events_number_a"], event_dict[ev]["events_number_b"]}')
    
    z_test(successes, trials, alpha_bonferroni)

Статистическое отличие в долях групп А и В имеют метрики product_page, просевшая на 12.93%, и purchase, просевная на 12.98%. Метрика product_cart, просевшая на 8.3%, статистически значимых отличий по группам не имеет.

## Общие выводы.
<a id="outcome"></a>

Участники первоначально набраны в нужном количестве. Но при этом мы должны учитывать, что в данных представленны люди, зарегистрировавшиеся после 17 декабря. Поскольку активность фактически завершилась к 31 декабря, люди, зарегистрировашиеся в период с 18 по 20 декабря, включительно, не могут быть учтены как полноценные участники теста.
С учётом этого фактора в группе А полноценно участвовали лишь 2747 человек, а в группе В - 928. Всего активных участников, прошедших тестирование в течение всех 14-ти дней - 3675 человек.

Распределение активности обеих групп близко к нормальному.
Активность в совершении событий для группы А несколько выше. Медианное значение событий на человека в контрольной группе - 6, в тестовой - 4. Максимальная статистически значимая величина количества событий на человека - 16 для группы А и 12 для группы В.
Количественно выбросов в группе В несколько больше, чем в контрольной группе.
Активность в целом у обеих групп одинаковая, но стоит помнить о большой разнице в численности групп (в 2.96 раза).


События по дням в группе А распределены близко к нормальному. В группе В - два пика, в начале и в середине теста. 30 декабря активность резко обрывается. В этот день активность (незначительную) показали только участники группы В.
Распределение по дням показывает бОльшую активность контрольной группы, что, судя по разнице графиков, не всегда может быть объяснено только значительным численным превосходством группы А.
14 декабря в активности контрольной группы произошёл всплеск, во многом определявшийся всплеском регистрации новых пользователей для этой группы.

Обращает на себя внимание, что в группе А товар в корзину положило меньше участников, чем совершило покупку. Это вызвано, тем, что здесь нет прямой последовательности.
С одной стороны, пользователь может использовать корзину вместо "Избранного", если оно вообще есть в данном случае. То есть, человек кладёт что-то в корзину, чтобы подумать, точно ли ему нужен этот продукт.
С другой стороны, покупатели могут воспользоваться возможностью "быстрой покупки", переходя сразу от товара к оплате. Скорее всего здесь мы сталкиваемся с людьми, которые уже точно знают, какой товар и с какими характеристиками они хотят приобрести.
В группе В залогинилось на 1 человека меньше, чем в тесте приняло активное участие. Кто-то совершал покупку без входа в личный кабинет или подобного действия.

При проведении тестирования необходимо учитывать сложный состав данных, которые нельзя назвать чистыми.
Во-первых, нужно учесть, что таблицы содержат информацию по двум тестам.
Во-вторых, данные по активности обрываются после 30 декабря, так что последние участники тестировались только 10-13 дней и их активность учитывать нельзя.
В-третьих, хотя вместо 6000 человек для этого А/В-теста набрано 6701, участвовало в тесте (совершали события) 3675 человек. То есть нужно учитывать, что совокупность зарегистрированных участников должна быть заметно выше необходимого минимума.
В-четвёртых, группы изначально отличаются по количеству очень сильно в 2.36 раза.
В-пятых, доля жителей Европы, привлечённых в тест в реальности составила 41.47% зарегистрированных  по 21 декабря пользователей из Европы, вместо оговоренных 15%, что превышает условие в ТЗ в 2.76 раза. Такое превышение влияет на репрезентативность теста и искажает его результаты.
В-шестых, 783 участника recommender_system_test участвовали и во втором тесте в тестовой группе, что может повлиять на результат нашего теста. Здесь стоит отметить, что пресечения по группам внутри recommender_system_test нет.
Наконец, данные указывают на совпадение теста по времени с двумя маркетинговыми событиями и с двумя праздниками: Рождеством (помним, что у нас большинство участников из региона Европейский Союз, добавляем к ним участников из Северной Америки) и Новым годом (что актуально на территории СНГ). Если Рождество в Европе - семейный праздник, то на период от Рождества до Трёх королей (6 января), а то и до 10 января, многие уезжают в отпуска для активного отдыха. 

Таким образом, ТЗ по разным причинам было нарушено  по шести пунктам. Длительность участия, количество участников, совпадение с другими маркетинговыми событиями и совпадение с праздниками влияют на результат критично. Тест необходимо было перенести.


На всех этапах воронки конверсия в группе В хуже, чем в группе А.
Вместо роста по всем метрикам на 10% получили падение метрик: по конверсии в product_page  -12.93%, по конверсии в product_cart  -8.3% и по конверсии в purchase  -12.98%.
Вместо заметного роста получили заметное падение.

Статистическое отличие в долях групп А и В имеют метрики product_page, просевшая на 12.93%, и purchase, просевная на 12.98%. Метрика product_cart, просевшая на 8.3%, статистически значимых отличий по группам не имеет.

Мы могли бы предположить, что внесённые изменения ухудшили продукт или усложнили пользовательский опыт. Однако мы должны учитывать, что по многим пунктам организаторам теста не удалось соблюсти не только условия ТЗ, но и общие принципы А/В-тестирования. Следовательно результаты теста НЕ могут быть признаны достоверными.